In [1]:
import sys
sys.path.append('../')
from config_sing import *

In [2]:
### import tools
import sqlite3
import itertools as it
from functools import reduce
from collections import defaultdict
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use("seaborn-whitegrid")

### Set Samples
fun = np.core.defchararray.add
idx = np.arange(1,6).astype("str")

INPUT    = reduce(fun, ["Input", idx             ])
INPUT20X = reduce(fun, ["Input", idx,     "_20x" ])
TFX_DMSO = reduce(fun, ["TFX",   idx[1:], "_DMSO"])
TFX_DEX  = reduce(fun, ["TFX",   idx[1:], "_Dex" ])
SAMPLES  = np.concatenate([INPUT, INPUT20X, TFX_DMSO, TFX_DEX])

### file path of database
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
FPATH_DB = os.path.join(fdiry, fname)
print(FPATH_DB)

/mount/work/out/proj_combeffect/database/fragment.db


In [4]:
fpath_db = f"file:{FPATH_DB}?mode=ro"
with sqlite3.connect(fpath_db, uri=True) as conn:
    ### set query: query all motifs binding to a given fragment
    query_out = f"""
       SELECT name FROM pragma_table_info('Fragment') 
        ;"""    

    ### query out motif bindings on the fragment
    query  = query_out
    cursor = conn.cursor()
    rows   = cursor.execute(query)

for row in rows:
    print(row)

('fragment',)
('chrom',)
('start',)
('end',)
('pct_at',)
('pct_gc',)
('num_A',)
('num_C',)
('num_G',)
('num_T',)
('num_N',)
('num_oth',)


In [7]:
fpath_db = f"file:{FPATH_DB}?mode=ro"
with sqlite3.connect(fpath_db, uri=True) as conn:
    query_out = f"""SELECT Frg.fragment FROM Fragment Frg;""" 
    query  = query_out
    cursor = conn.cursor()
    rows   = cursor.execute(query)

rows = it.islice(rows, 10)
for row in rows:
    frg = row[0]
    print(frg)

chr17_8148000_8148887
chr17_8148000_8148888
chr17_8148000_8148993
chr17_8148001_8148888
chr17_8148002_8148836
chr17_8148002_8149023
chr17_8148002_8149046
chr17_8148002_8149066
chr17_8148002_8149072
chr17_8148003_8148983


In [ ]:
def get_annot(frg, fpath_db=FPATH_DB, verbose=False):
    ### parse fragment info
    chrom, start, end = frg.split("_")
    
    fpath_db = f"file:{FPATH_DB}?mode=ro"
    with sqlite3.connect(fpath_db, uri=True) as conn:
        ### set query: query all motifs binding to a given fragment
        query_out = f"""
            SELECT Ant.binding
            FROM Annotation Ant
            WHERE Ant.fragment = '{frg}'
            ;"""    
        
        ### query out motif bindings on the fragment
        query  = query_out
        cursor = conn.cursor()
        rows   = cursor.execute(query)
    
        ### set query: query the corresponding motif info of each annotation
        fun  = lambda x, y: x + y
        mtfs = list(rows)
        mtfs = reduce(fun, mtfs)
        txt  = ','.join('?' * len(mtfs))

        query_out = f"""
            SELECT Mtf.motif, Mtf.score
            FROM Motif Mtf
            WHERE Mtf.binding IN ({txt})
            ;"""
    
        ### query out motif info
        query  = query_out
        cursor = conn.cursor()
        rows   = cursor.execute(query, mtfs)
    
    ### summarize the motif scores
    dct = defaultdict(lambda: 0)
    for idx, row in enumerate(rows):
        ### parse info
        motif, score = row

        ### sum the scores with same motifs
        dct[motif] += score
        
    if verbose:
        print(f"Fragment: {frg}; # Bindings: {idx+1}; # Motifs: {len(dct)}")
    
    return dct

In [ ]:
fpath_db = f"file:{fpath_db}?mode=ro"
with sqlite3.connect(fpath_db, uri=True) as conn:
    query_out = f"""
        SELECT Ant.fragment, Ant.binding, 
        FROM   Annotation Ant
        JOIN   Motif      Mtf ON Ant.binding  = Mtf.binding
        JOIN   Count      Cnt ON Ant.fragment = Cnt.fragment
        WHERE  Ant.fragment = '{frg}'
        ;""" 

    ### query out motif bindings on the fragment
    query  = query_out
    cursor = conn.cursor()
    rows   = cursor.execute(query)
        
    return cnames, rows

In [ ]:
def get_annot(frg, fpath_db=FPATH_DB, verbose=False):
    ### parse fragment info
    chrom, start, end = frg.split("_")
    
    with sqlite3.connect(fpath_db) as conn:
        ### set query: query all motifs binding to a given fragment
        query_out = f"""
            SELECT Ant.binding
            FROM Annotation Ant
            WHERE Ant.fragment = '{frg}'
            
            ;"""    
        
        ### query out motif bindings on the fragment
        query  = query_out
        cursor = conn.cursor()
        rows   = cursor.execute(query)
    
        ### set query: query the corresponding motif info of each annotation
        fun  = lambda x, y: x + y
        mtfs = list(rows)
        mtfs = reduce(fun, mtfs)
        txt  = ','.join('?' * len(mtfs))

        query_out = f"""
            SELECT Mtf.motif, Mtf.score
            FROM Motif Mtf
            WHERE Mtf.binding IN ({txt})
            ;"""
    
        ### query out motif info
        query  = query_out
        cursor = conn.cursor()
        rows   = cursor.execute(query, mtfs)
    
    ### summarize the motif scores
    dct = defaultdict(lambda: 0)
    for idx, row in enumerate(rows):
        ### parse info
        motif, score = row

        ### sum the scores with same motifs
        dct[motif] += score
        
    if verbose:
        print(f"Fragment: {frg}; # Bindings: {idx+1}; # Motifs: {len(dct)}")
    
    return dct